In [2]:
from lightning.pytorch.loggers import CSVLogger, WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from loaders.Mdndataloader import Modelnet40
from model.GNN_inf import Lightning_GNN
from sklearn.model_selection import train_test_split
import torch_geometric as tg
import numpy as np
import lightning as pl
import datetime
import yaml
import os
import torch

In [4]:
# Load array with params from config.yml
with open('config.yml', 'r') as f:
    config = yaml.safe_load(f)

# Data setup
dataset = Modelnet40(root=config['root'],
                     classes_yml='classes.yml',
                     split='test')

test_loader = tg.loader.DataLoader(dataset,
                                    batch_size=config['batch_size'],
                                    num_workers=4,
                                    shuffle=False)

# Model setup
GNN_model = Lightning_GNN(config=config)
GNN_model.to('cuda')
GNN_model.load_state_dict(torch.load('/home/lars/curr.ckpt')['state_dict'])

<All keys matched successfully>

In [5]:
GNN_model.to('cpu')

Lightning_GNN(
  (model): TransformerGNN(
    (enc1): Enc_block(
      (pconv): PointTrans_Layer(
        (attn): MLP(32, 32, 32)
        (pos): MLP(3, 32, 32)
        (conv): PointTransformerConv(3, 32)
      )
      (downlayer): PointTrans_Layer_down(
        (linear): Linear(in_features=32, out_features=32, bias=True)
        (down): Sequential(
          (0): Linear(in_features=32, out_features=32, bias=True)
          (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
      )
    )
    (enc2): Enc_block(
      (pconv): PointTrans_Layer(
        (attn): MLP(64, 64, 64)
        (pos): MLP(3, 64, 64)
        (conv): PointTransformerConv(32, 64)
      )
      (downlayer): PointTrans_Layer_down(
        (linear): Linear(in_features=64, out_features=64, bias=True)
        (down): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=Tru

In [6]:
# Test
trainer = pl.Trainer(max_epochs=1,
                     accelerator='cpu',
                     log_every_n_steps=1)

trainer.test(GNN_model,
            dataloaders=train_loader) 

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/lars/anaconda3/envs/PCloud/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/home/lars/anaconda3/envs/PCloud/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.8379032015800476
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc': 0.8379032015800476}]